# Building a SQL Agent

This notebook shows how to build a SQL agent that can transform phrases in natural language into SQL queries using Langchain and LLMs. This notebook is a modifed version of the examples provided in the Langchain documentation available here: https://python.langchain.com/docs/tutorials/sql_qa/

## Setup

### Setting up path folder

This section sets up the search path to include modules found in other folder of the project. If you put all your files in a single project shithis step is not needed. 

In [6]:
PARENT_FOLDER='notebooks'

import os, sys

this_path = os.getcwd()
folders = this_path.split('/')
index = folders.index(PARENT_FOLDER)
steps_to_root = len(folders) - folders.index(PARENT_FOLDER)
src_path = "/".join(['..'] * steps_to_root)

if src_path not in sys.path:
    sys.path.append(src_path)

### Importing libraries

In [7]:
from source.sql_agent import SQL_Agent, render
import warnings

### Initializing the agent
This section configures the database and initializes the agent by specifying the database, dialect and the LLM to use. The SQL_Agent library assumes you have created a .env file with the appropiate environment variables needed to connect to your LLM.

In [8]:
warnings.filterwarnings('ignore') # to supresss LangSmith warnings

db_uri="sqlite:///Chinook.db"
db_dialect = "SQLite"
my_agent = SQL_Agent(model_provider="openai",model_name="gpt-4o-mini",database_uri=db_uri, dialect=db_dialect, top_k=5)

## Execution

This question can be answered from a single table.

In [9]:
question = "What country has the biggest sales?"
render(my_agent.answer(question))

The country with the biggest sales is the **USA**, with total sales amounting to **$523.06**. Here are the top countries by sales:

1. **USA**: $523.06
2. **Canada**: $303.96
3. **France**: $195.10
4. **Brazil**: $190.10
5. **Germany**: $156.48

We modify the invocation to stream the answer and see how the Agent is thinking.

In [10]:
question = "What country has the biggest sales?"
render(my_agent.stream_answer(question))

================================ Human Message =================================

What country has the biggest sales?
================================== Ai Message ==================================
Tool Calls:
  sql_db_list_tables (call_XCDiNTE4fLFJnddXa6OQK5QI)
 Call ID: call_XCDiNTE4fLFJnddXa6OQK5QI
  Args:
================================= Tool Message =================================
Name: sql_db_list_tables

Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track
================================== Ai Message ==================================
Tool Calls:
  sql_db_schema (call_v2A9ma7UtaHabF2zjSL3l2ts)
 Call ID: call_v2A9ma7UtaHabF2zjSL3l2ts
  Args:
    table_names: Invoice
  sql_db_schema (call_txtdfUoVdJvPCJjqlLKOdAEo)
 Call ID: call_txtdfUoVdJvPCJjqlLKOdAEo
  Args:
    table_names: Customer
================================= Tool Message =================================
Name: sql_db_schema


CREATE TABLE "Customer" (
	"Customer

The country with the biggest sales is the **USA**, with total sales amounting to **$523.06**. Here are the top five countries with the highest sales:

1. **USA**: $523.06
2. **Canada**: $303.96
3. **France**: $195.10
4. **Brazil**: $190.10
5. **Germany**: $156.48

This question requires the use of 3 tables.

In [11]:
question = "What Album has sold the most tracks?"
render(my_agent.stream_answer(question))

================================ Human Message =================================

What Album has sold the most tracks?
================================== Ai Message ==================================
Tool Calls:
  sql_db_list_tables (call_h1x7B45x0kYQ0AZcf2Vx6xrs)
 Call ID: call_h1x7B45x0kYQ0AZcf2Vx6xrs
  Args:
================================= Tool Message =================================
Name: sql_db_list_tables

Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track
================================== Ai Message ==================================
Tool Calls:
  sql_db_schema (call_6G9QndJrInFAUz3KJEIQC6Pm)
 Call ID: call_6G9QndJrInFAUz3KJEIQC6Pm
  Args:
    table_names: Album
  sql_db_schema (call_rBu4g2fw1Oze2R9HMQTHnDLX)
 Call ID: call_rBu4g2fw1Oze2R9HMQTHnDLX
  Args:
    table_names: Track
  sql_db_schema (call_uCFv6QktVlQdFoCKxZ7uJ2at)
 Call ID: call_uCFv6QktVlQdFoCKxZ7uJ2at
  Args:
    table_names: InvoiceLine
==================

The album that has sold the most tracks is **"Minha Historia"** with a total of 27 tracks sold. Here are some other top-selling albums:

1. **"Greatest Hits"** - 26 tracks sold
2. **"Unplugged"** - 25 tracks sold
3. **"Acústico"** - 22 tracks sold
4. **"Greatest Kiss"** - 20 tracks sold

This cell shows the default template used by the agent. 

In [12]:
render(my_agent.prompt_template.messages[0].prompt.template)

You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most {top_k} results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the below tools. Only use the information returned by the below tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.

To start you should ALWAYS look at the tables in the database to see what you can query.
Do NOT skip this step.
Then you should query the schema of the most relevant tables.

This question requires 2 queries to be answered. 

In [13]:
question = "What is the average duration of the top 5 best selling tracks?"
render(my_agent.stream_answer(question))

================================ Human Message =================================

What is the average duration of the top 5 best selling tracks?
================================== Ai Message ==================================
Tool Calls:
  sql_db_list_tables (call_VNdrLjoTff2OQn8CbBW6ed8X)
 Call ID: call_VNdrLjoTff2OQn8CbBW6ed8X
  Args:
================================= Tool Message =================================
Name: sql_db_list_tables

Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track
================================== Ai Message ==================================
Tool Calls:
  sql_db_schema (call_69IQoat096p1Mi67x3qEgwGJ)
 Call ID: call_69IQoat096p1Mi67x3qEgwGJ
  Args:
    table_names: Track
  sql_db_schema (call_H6i0sSa1UJ4UtsXbRt0JXJuQ)
 Call ID: call_H6i0sSa1UJ4UtsXbRt0JXJuQ
  Args:
    table_names: InvoiceLine
================================= Tool Message =================================
Name: sql_db_schema


CREATE T

The average duration of the top 5 best selling tracks is approximately 268,338 milliseconds, which is about 4 minutes and 28 seconds.

This question requires 3 queries to be answered.

In [14]:
question = "What percent of customers bought tracks from the 2 albums with the higest sales?"
render(my_agent.stream_answer(question))

================================ Human Message =================================

What percent of customers bought tracks from the 2 albums with the higest sales?
================================== Ai Message ==================================
Tool Calls:
  sql_db_list_tables (call_jgQg3eMKa26onq7FnHHbPDkn)
 Call ID: call_jgQg3eMKa26onq7FnHHbPDkn
  Args:
================================= Tool Message =================================
Name: sql_db_list_tables

Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track
================================== Ai Message ==================================
Tool Calls:
  sql_db_schema (call_OlZREJr9zcBZgcO0GnIxndU9)
 Call ID: call_OlZREJr9zcBZgcO0GnIxndU9
  Args:
    table_names: Album, Invoice, InvoiceLine, Track
  sql_db_query (call_cHyhM395tvtFNa7dnv43qxrs)
 Call ID: call_cHyhM395tvtFNa7dnv43qxrs
  Args:
    query: SELECT AlbumId, SUM(Quantity) as TotalSales FROM InvoiceLine GROUP BY AlbumId ORDER

A total of 25 distinct customers purchased tracks from the two albums with the highest sales.

Now, to calculate the percentage of those customers compared to the total number of customers:

\[
\text{Percentage} = \left( \frac{\text{CustomersCount}}{\text{TotalCustomers}} \right) \times 100 = \left( \frac{25}{59} \right) \times 100 \approx 42.37\%
\]

Therefore, approximately **42.37%** of customers bought tracks from the two albums with the highest sales.

We include more specific instructions to make sure that sales are taking into account the price and not just the quantities.

In [16]:
question = "Defining sales as the product of quantity by unit price, what percent of customers have bought tracks from the 2 albums with the higest sales?"
render(my_agent.stream_answer(question))

================================ Human Message =================================

Defining sales as the product of quantity by unit price, what percent of customers have bought tracks from the 2 albums with the higest sales?
================================== Ai Message ==================================
Tool Calls:
  sql_db_list_tables (call_StOY84PZqkbbvieOvSu7wlBV)
 Call ID: call_StOY84PZqkbbvieOvSu7wlBV
  Args:
================================= Tool Message =================================
Name: sql_db_list_tables

Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track
================================== Ai Message ==================================
Tool Calls:
  sql_db_schema (call_xgVABVawuYoWLrpL3DxnTuhI)
 Call ID: call_xgVABVawuYoWLrpL3DxnTuhI
  Args:
    table_names: Album, Track, InvoiceLine, Customer
  sql_db_schema (call_f3wLQRyR7NJ0aPTPoQtKbzBk)
 Call ID: call_f3wLQRyR7NJ0aPTPoQtKbzBk
  Args:
    table_names: Invoice
=====

To determine the percentage of customers who have bought tracks from the two albums with the highest sales ('Battlestar Galactica (Classic), Season 1' and 'The Office, Season 3'), we found the following:

- Total number of customers: 59
- Number of customers who bought tracks from the top albums: 13

The percentage of customers who have purchased tracks from these albums is calculated as follows:

\[
\text{Percentage} = \left(\frac{\text{CustomersFromTopAlbums}}{\text{NumberOfCustomers}}\right) \times 100 = \left(\frac{13}{59}\right) \times 100 \approx 22.03\%
\]

Thus, approximately **22.03%** of customers have bought tracks from these two albums.

We modify the system prompt to include a definition of sales for any question.

In [17]:
new_system_message = '''System: You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct SQLite query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most 5 results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the below tools. Only use the information returned by the below tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.

To start you should ALWAYS look at the tables in the database to see what you can query.
Do NOT skip this step.
Then you should query the schema of the most relevant tables.
Sales are defined by the product of Quantity and Unit Price not just quantity.'''

my_agent.set_custom_system_message(new_system_message)
my_agent.initialize_agent()

We asked the same question without explicitely defining sales. 

In [18]:
question = "What percent of customers have bought tracks from the 2 albums with the higest sales?"
render(my_agent.stream_answer(question))

================================ Human Message =================================

What percent of customers have bought tracks from the 2 albums with the higest sales?
================================== Ai Message ==================================
Tool Calls:
  sql_db_list_tables (call_rEhpWjH4eaeD37bJO5o2e8ff)
 Call ID: call_rEhpWjH4eaeD37bJO5o2e8ff
  Args:
================================= Tool Message =================================
Name: sql_db_list_tables

Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track
================================== Ai Message ==================================
Tool Calls:
  sql_db_schema (call_T5NSEz0H9whCPcssH5phMOtW)
 Call ID: call_T5NSEz0H9whCPcssH5phMOtW
  Args:
    table_names: Album
  sql_db_schema (call_j1r6VJXTEaBMgc2r3h1IyIox)
 Call ID: call_j1r6VJXTEaBMgc2r3h1IyIox
  Args:
    table_names: Invoice
  sql_db_schema (call_DR6FTAPtQX4cVevYOoNyUi9W)
 Call ID: call_DR6FTAPtQX4cVevYOoNyUi9W
  Ar

The two albums with the highest sales are:

1. **Battlestar Galactica (Classic), Season 1** - $35.82
2. **The Office, Season 3** - $31.84

A total of **13 unique customers** purchased tracks from these two albums. There are **59 total customers** in the database.

To calculate the percentage of customers who purchased tracks from the two albums:

\[
\text{Percentage} = \left( \frac{\text{Unique Customers}}{\text{Total Customers}} \right) \times 100
= \left( \frac{13}{59} \right) \times 100 \approx 22.03\%
\]

Thus, approximately **22.03%** of customers have bought tracks from the two albums with the highest sales.